# Investigating if Fandango's Movie Ratings are Biased 
The project compares the Movie Rating by Fandango, an online movie ratings aggregator, and the actual movie ratings to determine if Fandango inflates the actual ratings as uncovered by a data journalist named Walt Hickey in 2015. The conclusions drawn by Hickey's research revealed the following insights from a sample fo Fandango's Movie Ratings, strongly suggesting that Fandango's Ratings are biased:

[image]

The goal of the project is to take the investigation further and analyze whether Fandango has corrected this bias present in their ratings and whether there has been any change in Fandango's rating system after Hickey's analysis

### The Data
We will be comparing the system's characteristics by using two different samples of data: one collected before Hickey's analysis and the other collected after Hickey's analysis from movies in 2016 and 2017.


In [18]:
# !pip install pandas
import pandas as pd

# Read and Explore the Dataset
prior_data = pd.read_csv('fandango_score_comparison.csv')
after_data = pd.read_csv('movie_ratings_16_17.csv')

prior_data.head(3)

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,...,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),74,86,66,7.1,7.8,5.0,4.5,3.70,4.3,...,3.90,3.5,4.5,3.5,3.5,4.0,1330,271107,14846,0.5
1,Cinderella (2015),85,80,67,7.5,7.1,5.0,4.5,4.25,4.0,...,3.55,4.5,4.0,3.5,4.0,3.5,249,65709,12640,0.5
2,Ant-Man (2015),80,90,64,8.1,7.8,5.0,4.5,4.00,4.5,...,3.90,4.0,4.5,3.0,4.0,4.0,627,103660,12055,0.5


In [19]:
after_data.head(3)

,movie,year,metascore,imdb,tmeter,audience,fandango,n_metascore,n_imdb,n_tmeter,n_audience,nr_metascore,nr_imdb,nr_tmeter,nr_audience
0,10 Cloverfield Lane,2016,76,7.2,90,79,3.5,3.80,3.60,4.5,3.95,4.0,3.5,4.5,4.0
1,13 Hours,2016,48,7.3,50,83,4.5,2.40,3.65,2.5,4.15,2.5,3.5,2.5,4.0
2,A Cure for Wellness,2016,47,6.6,40,47,3.0,2.35,3.30,2.0,2.35,2.5,3.5,2.0,2.5


### Isolate necessary columns from the datasets

We isolated the columns that offer information about Fandango's ratings in separate variables so we have just the data we're interested in more readily available for later use.
* For the data set with ratings previous to Hickey's analysis, select the following columns:  'FILM', 'Fandango_Stars', 'Fandango_Ratingvalue',
'Fandango_votes', 'Fandango_Difference'.
* For the other data set, select the following columns:  'movie', 'year', 'fandango'.


In [28]:
previous_data = prior_data[['FILM', 'Fandango_Stars', 'Fandango_Ratingvalue', 'Fandango_votes', 'Fandango_Difference']]

current_data = after_data[['movie', 'year', 'fandango']]

previous_data.head(3)

,FILM,Fandango_Stars,Fandango_Ratingvalue,Fandango_votes,Fandango_Difference
0,Avengers: Age of Ultron (2015),5.0,4.5,14846,0.5
1,Cinderella (2015),5.0,4.5,12640,0.5
2,Ant-Man (2015),5.0,4.5,12055,0.5


In [29]:
current_data.head(3)

,movie,year,fandango
0,10 Cloverfield Lane,2016,3.5
1,13 Hours,2016,4.5
2,A Cure for Wellness,2016,3.0
